<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [VAE1] - Variational AutoEncoder (VAE) with MNIST
<!-- DESC --> Episode 1 : Model construction and Training

<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Understanding and implementing a **variational autoencoder** neurals network (VAE)
 - Understanding a more **advanced programming model**

The calculation needs being important, it is preferable to use a very simple dataset such as MNIST to start with.

## What we're going to do :

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard

----
## Bug Note :
**Works in tf 2.0, but not in 2.2/2.3**  

See :
 - https://github.com/tensorflow/tensorflow/issues/34944  
 - https://github.com/tensorflow/probability/issues/519  

Bypass :
 - Use tf 2.0
 - Add `tf.config.experimental_run_functions_eagerly(True)` before compilation...  
Works fine in versions 2.2, 2.3 but with horrible perf. (7s -> 1'50s)
----

## Step 1 - Init python stuff

In [1]:
import numpy as np
import sys, importlib

import modules.vae
import modules.loader_MNIST

from modules.vae          import VariationalAutoencoder
from modules.loader_MNIST import Loader_MNIST

VariationalAutoencoder.about()


FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.28


## Step 2 - Get data

In [2]:
(x_train, y_train), (x_test, y_test) = Loader_MNIST.load()

Dataset loaded.
Normalized.
Reshaped to (60000, 28, 28, 1)


## Step 3 - Get VAE model
Nous allons instancier notre modèle VAE.  
Ce dernier est défini avec une classe python pour alléger notre code.  
La description de nos deux réseaux est donnée en paramètre.  
Notre modèle sera sauvegardé dans le dossier : ./run/<tag>

In [3]:
tag = 'MNIST.001'

input_shape = (28,28,1)
z_dim       = 2
verbose     = 1

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'}
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':1,  'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/MNIST.001

 ---------- Encoder -------------------------------------------------- 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 7, 64)     36928  

## Step 4 - Compile it

In [4]:
r_loss_factor = 1000

vae.compile( optimizer='adam', r_loss_factor=r_loss_factor)

Compiled.


## Step 5 - Train

In [5]:
batch_size        = 100
epochs            = 100
initial_epoch     = 0
k_size            = 1      # 1 mean using 100% of the dataset

In [ ]:
vae.train(x_train,
          x_test,
          batch_size        = batch_size, 
          epochs            = epochs,
          initial_epoch     = initial_epoch,
          k_size            = k_size
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 7s 111us/sample - loss: 63.3781 - vae_r_loss: 60.6616 - vae_kl_loss: 2.7165 - val_loss: 52.9564 - val_vae_r_loss: 49.0133 - val_vae_kl_loss: 3.9432
Epoch 2/100
60000/60000 [==============================] - 4s 67us/sample - loss: 50.9778 - vae_r_loss: 46.8340 - vae_kl_loss: 4.1438 - val_loss: 49.9606 - val_vae_r_loss: 45.9424 - val_vae_kl_loss: 4.0181
Epoch 3/100
60000/60000 [==============================] - 4s 67us/sample - loss: 48.7746 - vae_r_loss: 44.2006 - vae_kl_loss: 4.5740 - val_loss: 48.1807 - val_vae_r_loss: 43.4536 - val_vae_kl_loss: 4.7271
Epoch 4/100
60000/60000 [==============================] - 4s 67us/sample - loss: 47.7099 - vae_r_loss: 42.9376 - vae_kl_loss: 4.7722 - val_loss: 47.5955 - val_vae_r_loss: 42.5357 - val_vae_kl_loss: 5.0598
Epoch 5/100
60000/60000 [==============================] - 4s 67us/sample - loss: 47.1006 - vae_r_loss: 42.2272 - vae_kl_loss

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>